# chip-seq analysis

cut up the genome into sequences. align them to the genome. look at distribution.

## macs2 - peak caller

output is bedfile. has chromosome, start, strand information. this is where we come in.

We love [ENCODE](https://www.encodeproject.org). Encyclopedia of DNA Elements. This has lots of non-coding DNA elements (which is often regulatory).

How do we know if a DNA region is regulatory? Just because a protein binds to it does not necessarily mean it is biologically meaningful. 

We want to narrow down to TF ChIP-seq experiments.

For example, https://www.encodeproject.org/experiments/ENCSR000DRZ/

- BAM file has alignements.
- bigWig allows you to visualize data on genome browser (nice for presentations). Shush and Amber have been doing this with ChIP-seq peaks.

We are most interested in BED files. Want to use merged data (replicates 1,2 for example). Go with "conservative IDR thresholded peaks" (though there are many more).

In [ ]:
%load_ext nb_black

In [ ]:
# Stamatoyannopoulos - Univ of Washington
# CTCF ChIP-seq on human GM12878
# https://www.encodeproject.org/experiments/ENCSR000DRZ/
# conservative IDR thresholded peaks  1,2  hg19
!wget -N -nv --show-progress https://www.encodeproject.org/files/ENCFF963PJY/@@download/ENCFF963PJY.bed.gz

In [ ]:
!ls

In [ ]:
import pandas as pd

df = pd.read_csv("ENCFF963PJY.bed.gz", delimiter="\t", header=None)
df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

lengths = df.loc[:, 2] - df.loc[:, 1]
plt.hist(lengths, bins=20)
plt.title("Distribution of peak length")
plt.show()

## filter

### remove sequences > threshold length

### make sequences a constant size

get center of each peak and get window//2 on either side. window could be 200.


### filter using the functions above

In [ ]:
import pandas as pd

import chipseq_utils

In [ ]:
df = pd.read_csv("ENCFF963PJY.bed.gz", delimiter="\t", header=None)
df = chipseq_utils.filter_bed_by_max_length(df, max_length=250)
df = chipseq_utils.transform_bed_to_constant_size(df, window=250)
df.to_csv("ENCFF963PJY_filtered.bed.gz", sep="\t", index=False, header=False)

## something something fasta

### get reference genome

In this case, reference genome is in `.2bit` format, so we must convert to fasta.

In [ ]:
# get reference genome
!wget -N -nv --show-progress https://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit

In [ ]:
# get program that converts twobit to fasta format
!wget -N -nv --show-progress http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
!chmod +x twoBitToFa

In [ ]:
# convert 2bit for fasta.
reference_fasta = "hg19.fa.gz"
process = chipseq_utils.twobit_to_fasta("hg19.2bit", reference_fasta)
print(process)

In [ ]:
# Install bedtools if the program is not found.
![[ $(command -v bedtools) ]] || sudo apt-get install --yes --quiet bedtools

In [1]:
import chipseq_utils

In [5]:
# create fasta file from the bedfile.
# !bedtools getfasta -s -fi hg19.fa -bed ENCFF963PJY_filtered.bed.gz -fo ENCFF963PJY_filtered-hg19.fa

process = chipseq_utils.bedtools_getfasta(
    input_fasta="hg19.fa",
    output_fasta="ENCFF963PJY_filtered-hg19.fa",
    bed_file="ENCFF963PJY_filtered.bed.gz",
    force_strandedness=True,
)
print(process)

CompletedProcess(args=['bedtools', 'getfasta', '-s', '-fi', 'hg19.fa', '-bed', 'ENCFF963PJY_filtered.bed.gz', '-fo', 'ENCFF963PJY_filtered-hg19.fa'], returncode=0, stdout=b'', stderr=b'Warning: the index file is older than the FASTA file.\n')


In [6]:
!head -n 4 ENCFF963PJY_filtered-hg19.fa.gz

>chr6:91186482-91186732(.)
CGTTTGAAGGGGTATTTTCTGTTGTTCAATTGTGAACATATTTGGTTGCAGTTTTGAAGGCTTTTCCTCTAGAACTTTTCATGTCCTATTAGCTGATAACACTCCTGAGCATGAATGTCTCTGTGCTACTGCCCTCTAGGTGTGTTGGAAAAAATAGAGTGCTAATCCCACGCACCTGCTTTCAATTCGGCCCACATGAGTCTGGTGCCAGAGGTTTGTTCACTCTGAATATTTCTACGCAATCTTCAAA
>chr3:47841660-47841910(.)
catgcccgattaattgtgtatttttagtagagatggggtttctccatgttggtcaggctggtcttgaactcctgacctcaggtgacccgcctgcctgggcctccaaagtgctgggattacaggcatgagccaccacacctggccTTAGGTCTTGATTTATTTGTggctgggcatagtgggtcacacctgtaatctcaggagtttgggaggctgaagtgggaggatcgcttgaggccaggagtttgaggtc


In [7]:
import numpy as np

import chipseq_utils

In [24]:
a = chipseq_utils.one_hot(["TGCA"], alphabet="ACGT")
a.shape

(1, 4, 4)

In [27]:
nonsense

NameError: name 'nonsense' is not defined

In [30]:
>>> import numpy as np
>>> sequences = np.array(["AGGCCT", "GCTATTAN", "CGCTGC"])
>>> nonsense = chipseq_utils.get_nonsense_sequence_mask(sequences, nonsense_letters="N")
>>> nonsense
array([False,  True, False])
# >>> print(sequences[nonsense])
# ['GCTATTAN']
# >>> sequences[~nonsense]
# ['AGGCCT' 'CGCTGC']


NameError: name 'array' is not defined

In [29]:
nonsense

array([False,  True, False])

In [ ]:
>>> 
>>> nonsense = chipseq_utils.get_nonsense_sequence_mask(sequences, nonsense_letters="N")
>>> print(nonsense)
>>> print(sequences[nonsense])
>>> print(sequences[~nonsense])

In [ ]:
dd, ss = chipseq_utils.parse_fasta("ENCFF963PJY_filtered-hg19.fa")
nonsense = chipseq_utils.get_nonsense_sequence_mask(ss, nonsense_letters="N")
print(f"Found {nonsense.sum()} sequences with nonsense letters")
dd = dd[~nonsense]
ss = ss[~nonsense]
ss_one_hot = chipseq_utils.one_hot(ss)
print("Shape of one-hot encoded data:", ss_one_hot.shape)

In [ ]:
# Get GC content per sequence. This assumes that
# GC are in slices 1:3 of the one-hot encoded data.
# shape of this is (n_sequences,)
gc_content_pos = ss_one_hot[:, :, 1:3].any(-1).mean(1)

plt.hist(gc_content_pos, bins=20)
plt.title("Histogram of GC content among sequences")
plt.show()

## create negative data

Get non-overlap between positive peaks and negative peaks.

Negative labels sampled from same distribution but without the pattern we are interested in. We will use DNAseq for the same cell-type as our negative control. This gives us accessible regions.

In [ ]:
# Stamatoyannopoulos - Univ of Washington
# DNase-seq on human GM12878
# https://www.encodeproject.org/experiments/ENCSR000EMT/
!wget -N -nv --show-progress https://www.encodeproject.org/files/ENCFF097LEF/@@download/ENCFF097LEF.bed.gz

In [ ]:
!bedtools intersect -v -wa -a ENCFF097LEF.bed.gz -b ENCFF963PJY_filtered.bed.gz | gzip > neg_nonoverlap.bed.gz

In [ ]:
# TODO:
# do the same processing as above for negative peaks. in the end, you want
# one-hot representation of the negatives.

In [ ]:
import pandas as pd

df = pd.read_csv("neg_nonoverlap.bed.gz", delimiter="\t", header=None)
df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

lengths = df.loc[:, 2] - df.loc[:, 1]
plt.hist(lengths, bins=20)
plt.title("Distribution of peak length")
plt.show()

In [ ]:
df = filter_bed_by_max_length(df, max_length=250)
df = transform_bed_to_constant_size(df, window=250)
df.to_csv("neg_nonoverlap_filtered.bed.gz", sep="\t", index=False, header=False)

In [ ]:
# create fasta file from the bedfile.
!bedtools getfasta -s -fi hg19.fa -bed neg_nonoverlap_filtered.bed.gz -fo neg_nonoverlap_filtered-hg19.fa

In [ ]:
dd, ss = parse_fasta("neg_nonoverlap_filtered-hg19.fa")

nonsense_sequences = np.array(["N" in nt for nt in ss])
if nonsense_sequences.size > 0:
    dd = dd[~nonsense_sequences]
    ss = ss[~nonsense_sequences]

ss_one_hot_neg = one_hot_v1(ss, "ACGT")

In [ ]:
# Get GC content per sequence. This assumes that
# GC are in slices 1:3 of the one-hot encoded data.
# shape of this is (n_sequences,)
gc_content_neg = ss_one_hot_neg[:, :, 1:3].any(-1).mean(1)

plt.hist(gc_content_neg, bins=20)
plt.title("Histogram of GC content among sequences")
plt.show()

In [ ]:
# try to match positives and negatives by GC content.
# use the one-hot encoded array.
# we have to downsample negative peaks.
#
# do this after filtering with bedtools interset.
#
# then try to balance dataset. how much downsampling of negative data?

# also give labels of 0 or 1.

# save to hdf5

# and then train on a model, and look at saliency map.
# CTCF dataset is niiiiice. use that. Saliency map should show CTCF nicely.

In [ ]:
# https://datascience.stackexchange.com/questions/67645/how-to-resample-one-dataset-to-conform-to-the-distribution-of-another-dataset
# https://stackoverflow.com/questions/41495240/how-to-sample-data-based-off-the-distribution-of-another-dataset-in-r

In [ ]:
np.random.choice(gc_content_neg, replace=False, p=gc_content_pos)

In [ ]:
import scipy.stats

In [ ]:
kde = scipy.stats.kde.gaussian_kde(gc_content_pos)

In [ ]:
neg_chosen = np.random.choice(
    gc_content_neg, size=gc_content_pos.shape, replace=False, p=kde(gc_content_neg)
)

In [ ]:
s = ["ACGTGN"]

In [ ]:

def get_nonsense_sequence_mask(sequences, nonsense_letters="N"):
    """Return boolean array, where `True` marks a sequence containing nonsense letters."""
    sequences = np.asanyarray(sequences)
    return np.array(
        [any(letter in sequence for letter in nonsense_letters) for nt in ss]
    )


In [ ]:
[all(word in sentence for word in words) for sentence in strings]

In [ ]:
# TODO: generate about 10 datasets.